# 1) Chargement de la donnée

In [71]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [72]:
path  = "./../data/Villes/ville_1.csv"
ville = spark.read.format("csv").option("header", "true").load(path)

# 2) affichage

In [23]:
ville.show(2, vertical=True)

-RECORD 0----------------------------------
 id                 | 1                    
 vitesse_a_pied     | 0.12491350287937626  
 vitesse_a_velo     | 0.33310267434500335  
 home               | (lon:30.28 lat:19... 
 travail            | (lon:27.39 lat:15... 
 sportif            | False                
 casseur            | False                
 statut             | technicien_de_sur... 
 salaire            | 20326.98056600918    
 sexe               | H                    
 age                | 73                   
 sportivite         | 0.4163783429312542   
 velo_perf_minimale | 0.4                  
-RECORD 1----------------------------------
 id                 | 2                    
 vitesse_a_pied     | 4.85694978536844     
 vitesse_a_velo     | 12.951866094315841   
 home               | (lon:22.33 lat:25... 
 travail            | (lon:2.45 lat:17.53) 
 sportif            | True                 
 casseur            | False                
 statut             | employe   

# 3) exploration

# 3.1) types des colonnes

In [2]:
ville.dtypes

[('id', 'string'),
 ('vitesse_a_pied', 'string'),
 ('vitesse_a_velo', 'string'),
 ('home', 'string'),
 ('travail', 'string'),
 ('sportif', 'string'),
 ('casseur', 'string'),
 ('statut', 'string'),
 ('salaire', 'string'),
 ('sexe', 'string'),
 ('age', 'string'),
 ('sportivite', 'string'),
 ('velo_perf_minimale', 'string')]

# 3.2) décompte

In [24]:
ville.count()

1049

In [25]:
ville.distinct().count()

1049

## 3.3) première ligne

In [26]:
ville.first()

Row(id='1', vitesse_a_pied='0.12491350287937626', vitesse_a_velo='0.33310267434500335', home='(lon:30.28 lat:19.95)', travail='(lon:27.39 lat:15.92)', sportif='False', casseur='False', statut='technicien_de_surface', salaire='20326.98056600918', sexe='H', age='73', sportivite='0.4163783429312542', velo_perf_minimale='0.4')

# 4) manipulation

## 4.1) filtre, nombre d'hommes : where

In [28]:
ville.where(ville.sexe =="H").count()

493

## 4.2) requête sql : createOrReplaceTempView

In [7]:
# référencement de la dataFame commetable SQL
ville.createOrReplaceTempView("ville")
# requête sql
requete_sql = "select sexe, count(*) from ville group by sexe"
# utilisation de la variable spark pour appliquer la requête sql
spark.sql(requete_sql).collect()

[Row(sexe='F', count(1)=556), Row(sexe='H', count(1)=493)]

## 4.2.1) méthodes de l'objet dataFrame avec des équivalents SQL 

In [29]:
ville.groupBy("sexe").count().collect()

[Row(sexe='F', count=556), Row(sexe='H', count=493)]

## 4.3) moyenne des salaires par sexe

### 4.3.1) Gérer les types

In [32]:
# cette ligne ne fonctionne pas : il faut transformer le salaire en numérique
ville.groupBy(["sexe"]).mean("salaire").collect()

AnalysisException: '"salaire" is not a numeric column. Aggregation function can only be applied on a numeric column.;'

### 4.3.2) Spark transforme les string en double via le SQL

In [34]:
requete_sql = "select sexe, mean(salaire) from ville group by sexe"
spark.sql(requete_sql).collect()

[Row(sexe='F', avg(CAST(salaire AS DOUBLE))=23419.003849651763),
 Row(sexe='H', avg(CAST(salaire AS DOUBLE))=28727.584395358714)]

### 4.3.3) Spark propose des fonctions qui transforme aussi nativement les string en nombre

In [39]:
from pyspark.sql.functions import sum, avg, count, first
ville.groupBy(["sexe"]).agg(avg("salaire")).collect()

[Row(sexe='F', avg(salaire)=23419.003849651763),
 Row(sexe='H', avg(salaire)=28727.584395358714)]

### 4.3.4) sinon il faut gérer à la main

In [41]:
# on importe le type DoubleType
from pyspark.sql.types import DoubleType
# on rajoute une colonne "salaire_float" qui vaut la colonne "salaire" changée en DoubleType
ville = ville.withColumn("salaire_float", ville.salaire.cast(DoubleType() ))
# on peut ensuite calculer la moyenne sur cette nouvelle colonne "salaire_float"
ville.groupBy(["sexe"]).mean("salaire_float").collect()

[Row(sexe='F', avg(salaire_float)=23419.003849651763),
 Row(sexe='H', avg(salaire_float)=28727.584395358714)]

## 4.4) trouver le nombre de personne par tranche de salaire

In [44]:
categorie(34000)

30000

In [45]:

def categorie(salaire):
    """
    Calcule la dizaine de millier dans lequel se trouve le salaire.
    Ex : 
        15000 -> 10000
        34000 -> 30000
    """
    nb_de_dizaine_de_millier = float(salaire)//10000
    categorie                = 10000 * nb_de_dizaine_de_millier
    return  int(categorie)

# import de la fonction "udf" permettant d'enregistrer des fonctions définies par nous même, applicable sur des colonnes 
from pyspark.sql.functions import udf
# import des types sql, pour pouvoir type le résultat de notre fonction (Spark est codé en java, qui est un langage très typé)
from pyspark.sql.types     import *
# enregistrement de notre fonction comme une fonction udf
ma_fonction = udf(categorie , IntegerType())
# application de notre fonction udf sur la colonne salaire, et enregistrement du résultat dans une nouvelle colonne "salaire_categorie"
ville = ville.withColumn("salaire_categorie", ma_fonction("salaire"))
ville.show(1, vertical=True)

-RECORD 0----------------------------------
 id                 | 1                    
 vitesse_a_pied     | 0.12491350287937626  
 vitesse_a_velo     | 0.33310267434500335  
 home               | (lon:30.28 lat:19... 
 travail            | (lon:27.39 lat:15... 
 sportif            | False                
 casseur            | False                
 statut             | technicien_de_sur... 
 salaire            | 20326.98056600918    
 sexe               | H                    
 age                | 73                   
 sportivite         | 0.4163783429312542   
 velo_perf_minimale | 0.4                  
 salaire_float      | 20326.98056600918    
 salaire_categorie  | 20000                
only showing top 1 row



In [50]:
pop_par_tranche =ville.groupBy(["sexe", "salaire_categorie"]).count()
pop_par_tranche.orderBy("count").collect()

[Row(sexe='F', salaire_categorie=80000, count=1),
 Row(sexe='H', salaire_categorie=70000, count=2),
 Row(sexe='H', salaire_categorie=80000, count=2),
 Row(sexe='F', salaire_categorie=70000, count=3),
 Row(sexe='F', salaire_categorie=50000, count=4),
 Row(sexe='F', salaire_categorie=0, count=5),
 Row(sexe='H', salaire_categorie=60000, count=7),
 Row(sexe='H', salaire_categorie=50000, count=15),
 Row(sexe='F', salaire_categorie=40000, count=20),
 Row(sexe='H', salaire_categorie=40000, count=49),
 Row(sexe='F', salaire_categorie=30000, count=84),
 Row(sexe='H', salaire_categorie=30000, count=104),
 Row(sexe='H', salaire_categorie=10000, count=106),
 Row(sexe='F', salaire_categorie=20000, count=205),
 Row(sexe='H', salaire_categorie=20000, count=208),
 Row(sexe='F', salaire_categorie=10000, count=234)]

In [51]:
sorted(ville.groupBy(["sexe", "salaire_categorie"]).count().collect())

[Row(sexe='F', salaire_categorie=0, count=5),
 Row(sexe='F', salaire_categorie=10000, count=234),
 Row(sexe='F', salaire_categorie=20000, count=205),
 Row(sexe='F', salaire_categorie=30000, count=84),
 Row(sexe='F', salaire_categorie=40000, count=20),
 Row(sexe='F', salaire_categorie=50000, count=4),
 Row(sexe='F', salaire_categorie=70000, count=3),
 Row(sexe='F', salaire_categorie=80000, count=1),
 Row(sexe='H', salaire_categorie=10000, count=106),
 Row(sexe='H', salaire_categorie=20000, count=208),
 Row(sexe='H', salaire_categorie=30000, count=104),
 Row(sexe='H', salaire_categorie=40000, count=49),
 Row(sexe='H', salaire_categorie=50000, count=15),
 Row(sexe='H', salaire_categorie=60000, count=7),
 Row(sexe='H', salaire_categorie=70000, count=2),
 Row(sexe='H', salaire_categorie=80000, count=2)]

# 5) Group by : calcul de nombre d'élément par groupe

In [63]:
from pyspark.sql import functions as func

ville.groupBy("sexe").agg(func.size(func.collect_list('sexe'))).show()

+----+------------------------+
|sexe|size(collect_list(sexe))|
+----+------------------------+
|   F|                     556|
|   H|                     493|
+----+------------------------+



#  6) Group by : calcul de nombre d'élément distincts par groupe

In [58]:
ville.groupBy("sexe").agg(func.size(func.collect_set('salaire_categorie'))).show()

+----+------------------------------------+
|sexe|size(collect_set(salaire_categorie))|
+----+------------------------------------+
|   F|                                   8|
|   H|                                   8|
+----+------------------------------------+



In [9]:
path       = "./../data/Cyclistes/cycliste_1.csv" 
cycliste_1 = spark.read.format("csv").option("header", "true").load(path)

In [10]:
cycliste_1.take(3)

[Row(id='1', timestamp='1', sur_velo='False', velo='False', vitesse='0.12491350287937626', position='(lon:30.28 lat:19.95)', destination_finale='(lon:27.39 lat:15.92)'),
 Row(id='1', timestamp='2', sur_velo='False', velo='False', vitesse='0.12491350287937626', position='(lon:30.16 lat:19.97)', destination_finale='(lon:27.39 lat:15.92)'),
 Row(id='1', timestamp='3', sur_velo='False', velo='False', vitesse='0.12491350287937626', position='(lon:30.03 lat:19.99)', destination_finale='(lon:27.39 lat:15.92)')]

In [11]:
cycliste_1.count()

9

In [67]:
path       = "./../data/Cyclistes/*.csv" 
tous_les_cyclistes = spark.read.format("csv").option("header", "true").load(path)

In [69]:
tous_les_cyclistes.count()

42000

In [70]:
tous_les_cyclistes.columns

['id',
 'timestamp',
 'sur_velo',
 'velo',
 'vitesse',
 'position',
 'destination_finale']